In [ ]:
# import os
# import requests

# fname_scraper_API_key = 'scraper_API_key.txt' # TODO: change to your own API key
# if os.path.isfile(fname_scraper_API_key):
#     with open(fname_scraper_API_key, 'r') as fh:
#         scraper_API_key = fh.read()
# else:
#     print('Error: scraper API key file not found')
#     exit()

# payload = { 'api_key': scraper_API_key, 'url': 'https://www.amazon.de', 'retry_404': True, 'autoparse': True, 'country_code': 'eu', 'device_type': 'desktop' } 
# r = requests.get('https://api.scraperapi.com/', params=payload)
# print(r.text)


In [ ]:
import os
import numpy as np
import json
import http.client
import urllib.parse


def search_product(query: str, max_items: int=4, rapidAPI_key_index: int=0):
    fname_rapidAPI_keys = 'RAPIDAPI_KEYS.txt'
    RAPID_API_KEYS = []
    if os.path.isfile(fname_rapidAPI_keys):
        with open(fname_rapidAPI_keys, 'r') as fh:
            for key in fh:
                RAPID_API_KEYS.append(key.strip())
    else:
        print('Error: RAPIDAPI_KEYS.txt file not found')
    
    if rapidAPI_key_index >= len(RAPID_API_KEYS):
        print('Error: Too many shopping requests been made. Please wait for a while.')
        return None
    
    RapidAPI_host = "real-time-product-search.p.rapidapi.com"
    conn = http.client.HTTPSConnection(RapidAPI_host)

    headers = {
        'X-RapidAPI-Key': RAPID_API_KEYS[rapidAPI_key_index],
        'X-RapidAPI-Host': RapidAPI_host
    }

    formatted_query = urllib.parse.quote(query)
    conn.request("GET", f"/search?q={formatted_query}&country=ch&language=en", headers=headers)

    res = conn.getresponse()
    


    data = res.read().decode("utf-8")
    query_result = json.loads(data)

    if hasattr(query_result, 'message'):
        if query_result['message'][len('You have exceeded')] == 'You have exceeded':
            return search_product(query, max_items, rapidAPI_key=RAPID_API_KEYS[rapidAPI_key_index+1])
    
    if 'data' not in query_result:
        return None
    thumbnails = [query_result['data'][i]['product_photos'][0] for i in range(np.min((max_items, len(query_result['data']))))]
    links = [query_result['data'][i]['offer']['offer_page_url'] for i in range(np.min((max_items, len(query_result['data']))))]
    prices = [query_result['data'][i]['offer']['price'] for i in range(np.min((max_items, len(query_result['data']))))]
    prices = [price.replace('\xa0', ' ').replace(' + tax', '') for price in prices]
    return {'thumbnails': thumbnails, 'links': links, 'prices': prices, 'query': query}

In [ ]:
clothing_str_chatgpt = 'type of clothing,2-3 word description\nJacket,Winter Parka\nPants,Thermal Trousers\nBoots,Insulated Snow Boots\nGloves,Extreme Cold-Weather Gloves\nHat,Fleece-lined Beanie\nScarf,Thick Wool Scarf\nThermal Socks,Merino Wool Socks\nBase Layer,Long-sleeve Thermal Top\nOuter Layer,Waterproof Shell'
shopping_search_queries = clothing_str_chatgpt.split('\n')[1:]


In [ ]:
res0 = search_product(shopping_search_queries[0], max_items=2)

In [ ]:
res1 = search_product(shopping_search_queries[1], max_items=2)

In [ ]:
res2 = search_product(shopping_search_queries[2], max_items=2)


In [ ]:
res2
